# Presentation notes:
* Run this presentation with [`RISE`](https://github.com/damianavila/RISE): `conda install -c damianavila82 rise`
* Install [`nbextensions`](https://github.com/ipython-contrib/jupyter_contrib_nbextensions): `conda install -c conda-forge jupyter_contrib_nbextensions`
* Enable "Hide code" plugin with `jupyter nbextension enable hide_input/main`
* Click on the "Enter RISE Slideshow" button

In [ ]:
%%HTML
<style>
div.prompt {display: none}
div.cell {padding: 0px}
.slides{
    width: 95% !important;
    height: 95% !important;
    bottom: 0px;
    overflow-y: hidden !important;
    overflow-x: hidden !important;
}
</style>

In [ ]:
import adaptive
adaptive.notebook_extension()

# Import other modules that are used later
from functools import partial
import random
import holoviews as hv  # Plotting
from matplotlib import pyplot as plt
import numpy as np

def f(x, offset=0.07357338543088588):
    a = 0.01
    return x + a**2 / (a**2 + (x - offset)**2)

def plot_sharp_peak(figsize):
    plt.xkcd(randomness=0.5)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1, 1, 1)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    plt.xticks([])
    plt.yticks([])

    xs = np.linspace(0, 1, 300)
    ys = xs + 0.005**2 / (0.005**2 + (xs - 0.5)**2)

    plt.annotate(
        r'SHARP PEAK',
        xy=(0.5, 1.2), arrowprops=dict(arrowstyle='->'), xytext=(0.6, 0.8))

    plt.text(0.02, 1, r'$f(x) = x + \frac{a^2}{a^2 + (x-x_0)}$', fontdict={'size': 20});


    plt.plot(xs, ys)
    plt.title("1D example function")
    plt.xlabel('x')
    plt.ylabel('y')
    return fig


def plot_loss_interval(learner):
    if learner.npoints >= 2:
        x_0, x_1 = max(learner.losses, key=learner.losses.get)
        y_0, y_1 = learner.data[x_0], learner.data[x_1]
        plot = hv.Scatter(([x_0, x_1], [y_0, y_1]))
    else:
        plot = hv.Scatter([])
    return plot.opts(style=dict(size=6, color='r'))

learner = adaptive.Learner1D(f, bounds=(-1, 1))
plots = {0: learner.plot()}
for n in range(1, 101):
    xs, _ = learner.choose_points(1)
    learner.add_data(xs, map(learner.function, xs))
    plots[n] = (learner.plot() * plot_loss_interval(learner))[:, -1.1:1.1]
hm_plots = hv.HoloMap(plots, kdims=['npoints']).relabel('something smarter')
    
    
hom_learner = adaptive.Learner1D(f, bounds=(-1, 1), loss_per_interval=adaptive.learner.learner1D.uniform_loss)
hom_plots = {0: hom_learner.plot()}
for n in range(1, 101):
    xs, _ = hom_learner.choose_points(1)
    hom_learner.add_data(xs, map(hom_learner.function, xs))
    hom_plots[n] = (hom_learner.plot() * plot_loss_interval(hom_learner))[:, -1.1:1.1]
hm_hom_plots = hv.HoloMap(hom_plots, kdims=['npoints']).relabel('homogeneous')

<div>
  <img style="float: left; vertical-align:middle" src="presentation/logo-small.png">
  <h1><span style="">adaptive</span></h1>
</div>
a tool for adaptive and parallel evaluation of functions

– [github.com/python-adaptive/adaptive](https://github.com/python-adaptive/adaptive) <br>
– Bas Nijholt: [bas@nijho.lt](mailto:bas@nijho.lt)

<table>
  <tr>
    <th><img src="http://www.nijholt.org/images/profile.jpg" alt="Bas" height="200" width="200">Bas</th>
    <th><img src="http://joseph.weston.cloud/images/me_profile.jpg" alt="Joe" height="200" width="200">Joe</th>
    <th><img src="http://antonakhmerov.org/akhmerov_photo_fom.png" alt="Anton" height="200" width="200">Anton</th>
  </tr>
</table>

# This talk
* Who am I?
* Which problem are we solving?
* A motivating example
* Introducing the core concepts of `adaptive`
* Live coding example in the Jupyter notebook

# Who am I?
* Ph.D. student in Delft with Anton Akhmerov
* Theoretical computational quantum mechanics (Majorana hybrid semiconductor superconductor nanowire devices)

# Heavy numerics

Proof: cluster usage of last 60 days, I burned `0.5 yr/day` of CPU time
![](presentation/cluster.png)

Source: http://hpc05.quantumtinkerer.tudelft.nl/

![](presentation/example_plot1.png)

# The problems
* How to sample a function?
* When did we sufficiently sample the function?

# Simplest example, 1D function
We start with the most common use-case: sampling a 1D function $\ f: ℝ → ℝ$.

We will use the following function:

In [ ]:
plot_sharp_peak((8, 6));

## Homogeneous sampling, _the usual_
```python
xs = np.linspace(-1, 1, 100)
ys = f(xs)
```
_Strategy_: choose a point in the middle of the largest interval $\Delta$x

In [ ]:
%%output size=150
hm_hom_plots

## Better sampling, minimize the distance between the points
_Strategy_: choose a point in the middle of the largest interval $\sqrt{\Delta x^2 + \Delta y^2}$

In [ ]:
%%output size=120
hm_hom_plots + hm_plots

# the `adaptive` package
[`adaptive`](https://gitlab.kwant-project.org/qt/adaptive-evaluation) is an open-source package:
* 4314 lines of code and 628 commits
* first version released Feb 2018

**It does**:
* Smarter sampling of functions
    * 1D, 2D
    * random functions (0D)
    * numerical integration
* easy parallelization
* provide tools for live-plotting of the data

# The `learner` object

A learner takes the function to "learn" and the `bounds`.
```python
learner = adaptive.Learner1D(f, bounds=(-1, 1))
```

The three most important methods:

`loss = learner.loss()` "quality factor" how well do the points describe the function?

`xs, loss_improvements = learner.choose_points(n=10)` give me new points, and tell me how "good" the points are


`learner.add_point(x_new, y_new)` add the newly calculated data

# The `runner` object, _running the `learner`_

We basically want to to something like _(not really how it works internally)_:

```python
learner = adaptive.Learner1D(f, bounds=(-1, 1))
while learner.loss() > 0.01:
    xs, _ = learner.choose_points(4)  # we do nothing with the `loss_improvements` now
    for x in xs:
        y = learner.function(x)
        learner.add_point(x, y)
```

This has some problems: <br>
• Not using all the resources<br>
• Blocks the kernel, so we cannot plot the data while the calculation is in progress

The `Runner` solves these problems.
```python
learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)
```

# switch to the a notebook: [tutorial-notebook.ipynb](tutorial-notebook.ipynb)
![](presentation/live-coding-demos.jpg)

# Thank you!

### Install with
```bash
conda install -c conda-forge adaptive  # recommended
pip install adaptive
```
### Questions on
* Gitter chat [https://gitter.im/python-adaptive/adaptive](gitter.im/python-adaptive/adaptive)
* Github issues [https://github.com/python-adaptive/adaptive](github.com/python-adaptive/adaptive)

### This presentation
* [adaptive.nijho.lt](http://adaptive.nijho.lt)

![](presentation/example_plot2.png)

![](presentation/trump.gif)